In [ ]:
import logging
import csv
import re
import gzip
import pandas as pd

In [ ]:
# Input
movies_path = '../Data/input/schema_Movie.gz'
reviews_path= '../Data/input/schema_Review.gz'
inputlist = [movies_path,reviews_path]

# Output
moviereviewLog_path = '../Logs/movieReviews.log'
moviereviews_path = '../Data/input/moviereviews.csv'

logger = logging.getLogger()
fhandler = logging.FileHandler(filename=moviereviewLog_path, mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [ ]:
# csv form: NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING

taxo_pattern = re.compile("<http://schema.org/Review/description>|"\
                          "<http://schema.org/Review/reviewBody>|"\
                          "<http://schema.org/Review/reviewRating>|"\
                          "<http://schema.org/Rating/worstRating>|"\
                          "<http://schema.org/Rating/bestRating>|"\
                          "<http://schema.org/Rating/ratingValue>", re.IGNORECASE)
split_pattern = re.compile("^(_:.*)\s<(.*)>\s(.*)\s<(.*)>\s\.$", re.IGNORECASE)

url_lst = ['https://in.bookmyshow.com', 'https://thereviewmonk.com',
           'https://www.noopler.com', 'http://reviewschview.com',
           'https://www.flickfilosopher.com', 'https://www.rogerebert.com',
           'https://deepfocusreview.com', 'https://www.telegraph.co.uk',
           'https://nationalpost.com', 'https://www.imdb.com']

for input in inputlist:
    for url in url_lst:
        url_pattern = re.compile(url,re.IGNORECASE)
        with gzip.open(input,"rt") as f:
            i = 0
            detected = 0
            skipped = 0
            not_taxo = 0
            not_url = 0
            inserted = 0
            failed_updates = 0
            head = [next(f) for x in range(5000000)]
            # for line in f:
            for line in head:
                i += 1
                if not url_pattern.search(line):
                    not_url += 1
                    continue
                if not taxo_pattern.search(line):
                    not_taxo
                    continue
                match = split_pattern.match(line)
                if match is None:
                    skipped += 1
                    continue
                detected += 1
                props = match.groups()
                subj = props[0]
                predicate = props[1]
                obj = props[2]
                source = props[3]
                # Construct String
                update_query = "UPDATE MOVIEREVIEWS SET "
                params = [obj, subj, source]
                # print(line)
                if predicate.lower() == "http://schema.org/Review/reviewBody".lower():
                    update_query += "REVIEWBODY = ? "
                if predicate.lower() == "http://schema.org/Review/description".lower():
                    update_query += "REVIEWBODY = ? "
                if predicate.lower() == "http://schema.org/Review/reviewRating".lower():
                    update_query += "RATING = ? "
                if predicate.lower() == "http://schema.org/Rating/ratingValue".lower():
                    update_query += "REVIEWRATING = ? "
                if predicate.lower() == "http://schema.org/Rating/bestRating".lower():
                    update_query += "BESTRATING = ? "
                if predicate.lower() == "http://schema.org/Rating/worstRating".lower():
                    update_query += "WORSTRATING = ? "
                update_query += "WHERE NODE = ? AND URL = ?;"
                try:
                    # print(update_query, params)

                    inserted += 1
                except:
                    logging.debug("failed to execute for params " + str(props))
                    failed_updates += 1